## Extract HTTP

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

url = "https://pathumthani.moc.go.th/th/content/category/getarticles/id/114"
headers = {
    "Accept": "application/json, text/javascript, */*; q=0.01",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-GB,en-US;q=0.9,en;q=0.8",
    "Connection": "keep-alive",
    "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
    "Host": "pathumthani.moc.go.th",
    "Origin": "https://pathumthani.moc.go.th",
    "Referer": "https://pathumthani.moc.go.th/th/content/category/index/id/114",
    "Sec-Ch-Ua": '"Google Chrome";v="129", "Not;A=Brand";v="8", "Chromium";v="129"',
    "Sec-Ch-Ua-Mobile": "?0",
    "Sec-Ch-Ua-Platform": '"Windows"',
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36",
    "X-Requested-With": "XMLHttpRequest"
}

articles_data = []

In [ ]:
for page_index in range(1, 4):
    data = {
        'pageIndex': page_index 
    }
    
    response = requests.post(url, headers=headers, data=data)

    if response.status_code == 200:
        print(f"Request successful for page index: {page_index}:")
        data = response.json()
        html_content = data.get('html')
        soup = BeautifulSoup(html_content, 'html.parser')
        articles = soup.find_all('div', class_='inforow')
        
        for article in articles:
            link_tag = article.find('a', href=True)
            if link_tag:
                link = f"https://pathumthani.moc.go.th{link_tag['href']}"
            else:
                link = None
            
            date_tag = article.find('div', class_='image-date')
            if date_tag:
                date = date_tag.get_text(strip=True)
            else:
                date = None
            articles_data.append({'Link': link, 'Date': date})
    else:
        print(f"Request failed for page index {page_index} with status code: {response.status_code}")

In [11]:
df = pd.DataFrame(articles_data)
output_file = "articles_data.xlsx"
df.to_excel(output_file, index=False)

## Scraping 

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [3]:
def extract_table_data(soup):
    tables = soup.find_all('table')
    if len(tables) > 1:
        table = tables[1]
        rows = table.find_all('tr')[2:]
        data = []
        for row in rows:
            cols = row.find_all('td')
            cols = [ele.text.strip() for ele in cols]
            data.append(cols)
        return data
    else:
        return []

def clean_dataframe(df):
    num_columns = df.shape[1]
    column_names = ['ลำดับ', 'เลขที่ อก.', 'เลขทะเบียน', 'ชื่อโรงงาน', 'ที่อยู่', 'สถานะ']
    df.columns = column_names[:num_columns]
    if num_columns < len(column_names):
        df = df.drop(columns=column_names[num_columns:], errors='ignore')
    
    return df

def scrape_diw_schedule(url):
    response = requests.get(url)
    response.raise_for_status()

    soup = BeautifulSoup(response.content, 'html.parser')

    data = extract_table_data(soup)
    if not data:
        return None

    df = pd.DataFrame(data)
    if not df.empty:
        df = clean_dataframe(df)
    return df


In [4]:
if __name__ == "__main__":
    base_url = "http://reg.diw.go.th/job4/report_prov.asp?prov=13&page="
    all_dfs = []

    for page_num in range(1, 30):
        url = base_url + str(page_num)
        try:
            df = scrape_diw_schedule(url)
            if df is not None:
                all_dfs.append(df)
                print(f"Scraped page {page_num} successfully!")
            else:
                print(f"No data on page {page_num}")
        except Exception as e:
            print(f"Error scraping page {page_num}: {e}")

    final_df = pd.concat(all_dfs, ignore_index=True)

Scraped page 1 successfully!
Scraped page 2 successfully!
Scraped page 3 successfully!
Scraped page 4 successfully!
Scraped page 5 successfully!
Scraped page 6 successfully!
Scraped page 7 successfully!
Scraped page 8 successfully!
Scraped page 9 successfully!
Scraped page 10 successfully!
Scraped page 11 successfully!
Scraped page 12 successfully!
Scraped page 13 successfully!
Scraped page 14 successfully!
Scraped page 15 successfully!
Scraped page 16 successfully!
Scraped page 17 successfully!
Scraped page 18 successfully!
Scraped page 19 successfully!
Scraped page 20 successfully!
Scraped page 21 successfully!
Scraped page 22 successfully!
Scraped page 23 successfully!
Scraped page 24 successfully!
Scraped page 25 successfully!
Scraped page 26 successfully!
Scraped page 27 successfully!
Scraped page 28 successfully!
Scraped page 29 successfully!


In [5]:
final_df_cleaned = final_df.drop(columns=['ลำดับ', 'เลขที่ อก.'])
final_df_cleaned['ที่อยู่'] = final_df_cleaned['ที่อยู่'].str.replace(r'\r\n|\t', '', regex=True).str.strip()
final_df_cleaned['สถานะ'] = final_df_cleaned['สถานะ'].str.replace(r'\r\n|\t', '', regex=True).str.strip()
final_df_cleaned

,เลขทะเบียน,ชื่อโรงงาน,ที่อยู่,สถานะ
0,จ3-50(4)-1/40ปท,บริษัท บัญชากิจ จำกัด,71 ม.14 ซ.-ถ.- ต.ลำไทรอ.ลำลูกกา จ.ปทุมธานี,ไม่เข้าข่ายเนื่องจากผลิตแอสฟัลท์ติกคอนกรีตพิจา...
1,3-50(4)-1/43ปท,บริษัท ซีอาร์เอ กรุ๊ป จำกัด,38 ม.10ถ.วงแหวนตะวันตกสาย 37 ต.คูบางหลวงอ.ลาดห...,ไม่เข้าข่ายเนื่องจากผลิตแอสฟัลท์ติกคอนกรีตพิจา...
2,3-50(4)-1/31ปท,บริษัท พรีเมียลูบริแค้นท์ จำกัด,28/13 ม.3 ซ.ใจเอื้อถ.กรุงเทพ-ปทุมธานี ต.บางขะแ...,ผ่านเกณฑ์ ดีพิจารณาเสร็จ 9/1/2557 มีข้อแนะนำ...
3,3-50(4)-1/16ปท,บริษัท สยามลูบริแค้นท์อินดัสทรี่ จำกัด,28/8 ม.3 ซ.ใจเอื้อถ.กรุงเทพ-ปทุมธานี ต.บางขะแย...,ผ่านเกณฑ์ ดีพิจารณาเสร็จ 12/2/2557 มีข้อแนะน...
4,จ3-45(1)-1/37ปท,บริษัท ลีน่า (ประเทศไทย) จำกัด,40 ม.4ถ.รังสิต-นครนายก ต.บึงบอนอ.หนองเสือ จ.ปท...,ผ่านเกณฑ์ ดีพิจารณาเสร็จ 24/6/2557
...,...,...,...,...
575,จ3-42(1)-9/47ปท,บริษัท อินโน-คอนส์ (ประเทศไทย) จำกัด,22/7 ม.7 ต.คลองสี่อ.คลองหลวง จ.ปทุมธานี,ผ่านเกณฑ์ ดีพิจารณาเสร็จ 2/12/2557
576,3-7(4)-1/22ปท,บริษัท น้ำมันพืชปทุม จำกัด,29/3 ม.6 ซ.-ถ.ปทุมธานี-ลาดหลุมแก้ว ต.คูบางหลวง...,ผ่านเกณฑ์ ดีพิจารณาเสร็จ 18/10/2559 ขอขยายโร...
577,จ3-42(1)-13/58ปท,บริษัท กู๊ดไทม์ เมทัล เพสท์ จำกัด,52/1 ม.3 ต.คลองควายอ.สามโคก จ.ปทุมธานี,ผ่านเกณฑ์ ดีพิจารณาเสร็จ 24/3/2559
578,จ3-91(2)-1/44ปท,บริษัท ไทยสเปเชี่ยลแก๊ส จำกัด,100/38 ม.1 ต.สามโคกอ.สามโคก จ.ปทุมธานี,ผ่านเกณฑ์ ดีพิจารณาเสร็จ 3/6/2559 มี CD


In [49]:
final_df_cleaned.to_csv('ข้อมูลโรงงานที่ประเมินความเสี่ยงจังหวัดปทุมธานี.csv', index=False, encoding='utf-8-sig')